# Deep Sequence Modelling
We will go through some techniques of modelling sequences with Deep Learning models, namely we want to predict sentiment of IMDB movie reviews.

### RNN  

<img src="rnn.png" width="20%">

A recurrent neural network (RNN) is a class of artificial neural network where connections between units form a directed cycle. This creates an internal state of the network which allows it to exhibit dynamic temporal behavior. It is the simplest sequence model and hence we will start with this one, in Keras this model is defined as:

```python
keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, 
                                 kernel_initializer='glorot_uniform', 
                                 recurrent_initializer='orthogonal', 
                                 bias_initializer='zeros', 
                                 kernel_regularizer=None, 
                                 recurrent_regularizer=None, 
                                 bias_regularizer=None, 
                                 activity_regularizer=None, 
                                 kernel_constraint=None, recurrent_constraint=None, 
                                 bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)
```

#### Arguments:

<ul>
<li><strong>units</strong>: Positive integer, dimensionality of the output space.</li>
<li><strong>activation</strong>: Activation function to use
    (see <a href="http://keras.io/activations/">activations</a>).
    If you pass None, no activation is applied
    (ie. "linear" activation: <code>a(x) = x</code>).</li>
<li><strong>use_bias</strong>: Boolean, whether the layer uses a bias vector.</li>
<li><strong>kernel_initializer</strong>: Initializer for the <code>kernel</code> weights matrix,
    used for the linear transformation of the inputs.
    (see <a href="https://keras.io/initializers/">initializers</a>).</li>
<li><strong>recurrent_initializer</strong>: Initializer for the <code>recurrent_kernel</code>
    weights matrix,
    used for the linear transformation of the recurrent state.
    (see <a href="https://keras.io/initializers/">initializers</a>).</li>
<li><strong>bias_initializer</strong>: Initializer for the bias vector
    (see <a href="https://keras.io/initializers/">initializers</a>).</li>
<li><strong>kernel_regularizer</strong>: Regularizer function applied to
    the <code>kernel</code> weights matrix
    (see <a href="https://keras.io/regularizers/">regularizer</a>).</li>
<li><strong>recurrent_regularizer</strong>: Regularizer function applied to
    the <code>recurrent_kernel</code> weights matrix
    (see <a href="https://keras.io/regularizers/">regularizer</a>).</li>
<li><strong>bias_regularizer</strong>: Regularizer function applied to the bias vector
    (see <a href="https://keras.io/regularizers/">regularizer</a>).</li>
<li><strong>activity_regularizer</strong>: Regularizer function applied to
    the output of the layer (its "activation").
    (see <a href="https://keras.io/regularizers/">regularizer</a>).</li>
<li><strong>kernel_constraint</strong>: Constraint function applied to
    the <code>kernel</code> weights matrix
    (see <a href="https://keras.io/constraints/">constraints</a>).</li>
<li><strong>recurrent_constraint</strong>: Constraint function applied to
    the <code>recurrent_kernel</code> weights matrix
    (see <a href="https://keras.io/constraints/">constraints</a>).</li>
<li><strong>bias_constraint</strong>: Constraint function applied to the bias vector
    (see <a href="https://keras.io/constraints/">constraints</a>).</li>
<li><strong>dropout</strong>: Float between 0 and 1.
    Fraction of the units to drop for
    the linear transformation of the inputs.</li>
<li><strong>recurrent_dropout</strong>: Float between 0 and 1.
    Fraction of the units to drop for
    the linear transformation of the recurrent state.</li>
</ul>

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import keras 

import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing import image

from keras.datasets import imdb
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN

from keras.layers import Activation, TimeDistributed, RepeatVector
from keras.callbacks import EarlyStopping, ModelCheckpoint

## IMDB sentiment classification task

This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. 

IMDB provided a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. 

There is additional unlabeled data for use as well. Raw text and already processed bag of words formats are provided. 

http://ai.stanford.edu/~amaas/data/sentiment/

### Data Preparation - IMDB

In [ ]:
max_features = 20000
maxlen = 100  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print("Loading data...")
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Example:')
print(X_train[:1])

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

#### Model building 

First start by building a vanilla RNN

In [ ]:
### BUILD VANILLA RNN
### EPOCHS CAN BE SLOW SO FOR NOW SET NR_EPOCHS TO 1
hidden_size = 128
output_size = 1
dropout_ratio = 0.2

model = Sequential()
model.add(Embedding(max_features, hidden_size, input_length=maxlen))


#### IMPLEMENT HERE


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, epochs=1, 
          validation_data=(X_test, y_test))

loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("Test set loss is: %s" % loss)
print("Test set accuracy is: %s" % acc)

The results are acceptable but we can probably do better by using more advanced sequence models. Let's look at the LSTM first.

### LSTM  

A LSTM network is an artificial neural network that contains LSTM blocks instead of, or in addition to, regular network units. A LSTM block may be described as a "smart" network unit that can remember a value for an arbitrary length of time. 

Unlike traditional RNNs, an Long short-term memory network is well-suited to learn from experience to classify, process and predict time series when there are very long time lags of unknown size between important events.

<img src="gru.png" width="60%">

### GRU  

Gated recurrent units are a gating mechanism in recurrent neural networks and an efficient alternative to LSTMs. 

They are very similar to the way LSTMs work but they have fewer parameters than LSTM, as they lack an output gate. Still, in theory, they are able to model long-term dependencies. Since we do not have access to a GPU we are going to use this one.

```python
keras.layers.recurrent.GRU(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, 
                           kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
                           bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, 
                           bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, 
                           recurrent_constraint=None, bias_constraint=None, 
                           dropout=0.0, recurrent_dropout=0.0)
```

## GRU

In [ ]:
### IMPLEMENT GRU
hidden_size = 128
output_size = 1
dropout_ratio = 0.2

model = Sequential()
model.add(Embedding(max_features, hidden_size, input_length=maxlen))


#### IMPLEMENT HERE


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          epochs=1, validation_data=(X_test, y_test))
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("Test set loss is: %s" % loss)
print("Test set accuracy is: %s" % acc)

What do you notice compared to the results of the vanilla RNN?

## Convolutional GRU

This section demonstrates the use of a **Convolutional GRU network**.

We can use a convolutional layer before the GRU layer to reduce the input size and speed up training.
We get the following architecture:

1. Embedding layer
2. Dropout layer
3. Convolutional layer (Relu)
4. Max Pooling Layer
5. GRU Cell
6. Fully Connected layer (Sigmoid)


In [ ]:
from keras.layers.convolutional import Conv1D, MaxPooling1D

In [ ]:
### IMPLEMENT CONVOLUTIONAL LSTM
### REMEMBER WE ARE WORKING WITH 1D CONVOLUTIONAL LAYERS HERE
hidden_size = 128
output_size = 1
dropout_ratio = 0.2
nr_filters = 64
filter_size = 5
pool_size = 4

model = Sequential()
model.add(Embedding(max_features, hidden_size, input_length=maxlen))


#### IMPLEMENT HERE


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, 
          epochs=1, validation_data=(X_test, y_test))

loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("Test set loss is: %s" % loss)
print("Test set accuracy is: %s" % acc)

If everything went correctly, the model trained significantly faster and obtained a higher accuracy, awesome! 

## Convolutional sequence model

We can also fit only a convolutional model to the sequence. We get the following architecture:

1. Embedding layer
2. Dropout layer
3. Convolutional layer (Relu)
4. Max Pooling Layer
5. Flatten Layer
6. Fully Connected layer (Sigmoid)

In [ ]:
### IMPLEMENT CONVOLUTIONAL SEQUENCE MODEL
hidden_size = 128
output_size = 1
dropout_ratio = 0.2
nr_filters = 64
filter_size = 5
pool_size = 4

model = Sequential()
model.add(Embedding(max_features, hidden_size, input_length=maxlen))


#### IMPLEMENT HERE


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
model.fit(X_train, y_train, batch_size=batch_size, 
          epochs=1, validation_data=(X_test, y_test))

loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("Test set loss is: %s" % loss)
print("Test set accuracy is: %s" % acc)

Removing the LSTM cell increases the training speed significantly again while preserving performance levels! What else is catching your attention here?